In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# YOLO

In [ ]:
!pip install ultralytics

In [ ]:
import os
import cv2
import requests
from osgeo import gdal
import geopandas as gpd
from shapely.geometry import Point, box
from tqdm import tqdm
from ultralytics import YOLO
from io import BytesIO

# Load the trained YOLO model
checkpoint_path = '/content/drive/MyDrive/aerial_image_recognition/img/train/Tokyo/yolov8_tokyo_checkpoint.pt'
model = YOLO(checkpoint_path)
print("Model loaded from checkpoint successfully!")

# Load the 'ramki.shp' layer using Geopandas
ramki_shp_path = '/content/drive/MyDrive/aerial_image_recognition/gis/shp/ramki.shp'
ramki_gdf = gpd.read_file(ramki_shp_path)

# Select the specific area named 'srodmiescie' (adjust this filter for your case)
srodmiescie_gdf = ramki_gdf[ramki_gdf['name'] == 'srodmiescie']

# Get the bounding box of 'srodmiescie'
minx, miny, maxx, maxy = srodmiescie_gdf.total_bounds
print(f"Bounding box of srodmiescie: {minx}, {miny}, {maxx}, {maxy}")

# Define WMS parameters
wms_url = "https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/HighResolution"
wms_params = {
    'service': 'WMS',
    'version': '1.3.0',
    'request': 'GetMap',
    'layers': 'ORTO',
    'styles': '',
    'crs': 'EPSG:3857',  # Match to your projected CRS (Web Mercator)
    'bbox': f'{minx},{miny},{maxx},{maxy}',  # Update for each tile
    'width': 1200,  # Set the image size matching your model
    'height': 1200,
    'format': 'image/tiff'
}

# Define an empty GeoDataFrame for storing car/truck detections
car_centroids_gdf = gpd.GeoDataFrame(columns=['geometry', 'class', 'confidence'])

# Loop through and fetch tiles from the WMS server
stride = 600  # Adjust stride for partial overlap
tile_size = 1200

# Split the area into smaller tiles based on stride and tile size
x_coords = list(range(int(minx), int(maxx), stride))
y_coords = list(range(int(miny), int(maxy), stride))

total_tiles = len(x_coords) * len(y_coords)

with tqdm(total=total_tiles, desc="Processing Tiles") as pbar:
    for x in x_coords:
        for y in y_coords:
            # Adjust bounding box for each tile
            tile_bbox = f'{x},{y},{x+tile_size},{y+tile_size}'
            wms_params['bbox'] = tile_bbox

            # Request the tile from WMS
            response = requests.get(wms_url, params=wms_params)

            if response.status_code == 200:
                # Load the image into OpenCV
                img_data = BytesIO(response.content)
                img = gdal.Open(img_data).ReadAsArray()
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

                # Run YOLO inference on the tile
                results = model(img)

                # Extract bounding boxes and append to GeoDataFrame
                for box in results[0].boxes:
                    if box.cls in [0, 1] and box.conf > 0.4:  # Only consider cars/trucks
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        centroid_x = (x1 + x2) / 2 + x  # Adjust based on the tile's position
                        centroid_y = (y1 + y2) / 2 + y

                        # Append the result to the GeoDataFrame
                        car_centroids_gdf = car_centroids_gdf.append({
                            'geometry': Point(centroid_x, centroid_y),
                            'class': 'Car' if box.cls == 0 else 'Truck',
                            'confidence': box.conf.item()
                        }, ignore_index=True)

            pbar.update(1)

# Set CRS to match the WMS (EPSG:3857 for Web Mercator)
car_centroids_gdf.set_crs(epsg=3857, inplace=True)

# Save the results to a GeoJSON file
output_geojson_path = '/content/drive/MyDrive/aerial_image_recognition/gis/car_centroids_2.geojson'
car_centroids_gdf.to_file(output_geojson_path, driver='GeoJSON')

print(f"Car and truck centroids saved to {output_geojson_path}")


# ONNX

## CPU

In [ ]:
!pip install onnxruntime-gpu
!pip install owslib
!pip install geopandas
!pip install pyproj
!pip install tqdm
!pip install dnspython

In [ ]:
# Check PyTorch CUDA availability
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)

# Check NVIDIA driver and CUDA version
!nvidia-smi

# Check ONNX providers
import onnxruntime as ort
print("\nAvailable ONNX providers:", ort.get_available_providers())
print("Current provider options:", ort.get_device())

In [ ]:
import onnxruntime as ort
import numpy as np
import cv2
from owslib.wms import WebMapService
import geopandas as gpd
from shapely.geometry import Point, box
import io
from PIL import Image
import os
from google.colab import drive
from tqdm import tqdm
import time
import gc
import logging
import math
from pyproj import Transformer
from shapely.ops import transform
import pyproj

# Configuration settings
CONFIG = {
    'wms_url': "https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/StandardResolution",
    'tile_size_meters': 64.0,      # 50m tiles
    'pixel_size': 0.10,          # 10cm/pixel
    'model_input_size': 640,     # Model input size
    'confidence_threshold': 0.4,  # Lower confidence threshold (was 0.5)
    'tile_overlap': 0.2,         # 20% overlap between tiles
    'duplicate_distance_threshold': 2e-5  # For removing duplicates from overlapping areas
}

def setup_logging(base_dir):
    """Minimal logging configuration - file only, no console output"""
    log_path = os.path.join(base_dir, 'detection_debug.log')

    # Suppress all logging to console
    logging.getLogger().handlers = []

    # File-only handler
    file_handler = logging.FileHandler(log_path, mode='w')
    file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

    # Root logger setup
    root_logger = logging.getLogger()
    root_logger.addHandler(file_handler)
    root_logger.setLevel(logging.INFO)

    # Suppress specific loggers that might be noisy
    logging.getLogger('owslib').setLevel(logging.WARNING)
    logging.getLogger('urllib3').setLevel(logging.WARNING)

    return log_path

def setup_environment():
    """Setup paths and mount Google Drive"""
    drive.mount('/content/drive', force_remount=True)
    base_dir = "/content/drive/Othercomputers/My_laptop/car_recognition"

    # Setup logging first
    log_path = setup_logging(base_dir)

    return {
        'model_path': os.path.join(base_dir, "models/car_aerial_detection_yolo7_ITCVD_deepness.onnx"),
        'frame_path': os.path.join(base_dir, "gis/shp/frames/warsaw.shp"),
        'output_path': os.path.join(base_dir, 'gis/shp/detection_results/warsaw.geojson'),
        'tiles_path': os.path.join(base_dir, 'gis/shp/processing_tiles_warsaw.geojson'),
        'log_path': log_path
    }

def meters_to_degrees(meters, latitude):
    """Convert meters to degrees at a given latitude"""
    earth_radius = 6378137
    degrees_longitude = meters / (earth_radius * math.cos(math.radians(latitude)) * 2 * math.pi / 360)
    degrees_latitude = meters / (earth_radius * 2 * math.pi / 360)
    return degrees_longitude, degrees_latitude

def get_tile_bboxes(bbox, tile_size_meters):
    """Generate overlapping tile bounding boxes"""
    minx, miny, maxx, maxy = bbox

    # Calculate tile size in degrees at the middle latitude
    mid_lat = (miny + maxy) / 2
    tile_size_lon, tile_size_lat = meters_to_degrees(tile_size_meters, mid_lat)

    # Calculate overlap in degrees
    overlap_lon = tile_size_lon * CONFIG['tile_overlap']
    overlap_lat = tile_size_lat * CONFIG['tile_overlap']

    # Calculate step size (tile size minus overlap)
    step_lon = tile_size_lon * (1 - CONFIG['tile_overlap'])
    step_lat = tile_size_lat * (1 - CONFIG['tile_overlap'])

    tile_bboxes = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            tile_bbox = (
                x,
                y,
                min(x + tile_size_lon, maxx),
                min(y + tile_size_lat, maxy)
            )
            tile_bboxes.append(tile_bbox)
            y += step_lat
        x += step_lon

    return tile_bboxes

def get_wms_image(wms, bbox):
    """Simplified WMS image retrieval without logging"""
    width_pixels = height_pixels = CONFIG['model_input_size']

    for attempt in range(3):
        try:
            img = wms.getmap(
                layers=['Raster'],
                srs='EPSG:4326',
                bbox=bbox,
                size=(width_pixels, height_pixels),
                format='image/png',
                transparent=True
            )

            return Image.open(io.BytesIO(img.read())).convert('RGB')

        except Exception as e:
            if attempt == 2:
                raise
            time.sleep(1)

def analyze_model(session):
    """Analyze ONNX model structure and outputs"""
    # Get model inputs
    inputs = session.get_inputs()
    outputs = session.get_outputs()

    logging.info("\nModel Analysis:")
    logging.info("Input details:")
    for input in inputs:
        logging.info(f"Name: {input.name}")
        logging.info(f"Shape: {input.shape}")
        logging.info(f"Type: {input.type}")

    logging.info("\nOutput details:")
    for output in outputs:
        logging.info(f"Name: {output.name}")
        logging.info(f"Shape: {output.shape}")
        logging.info(f"Type: {output.type}")

    # The number of classes can be inferred from output shape
    # YOLO output shape is typically [batch, num_boxes, num_classes + 5]
    num_classes = outputs[0].shape[2] - 5  # subtract 5 for box coords + confidence
    logging.info(f"\nNumber of classes: {num_classes}")

    return num_classes

def process_detections(outputs, tile_bbox, num_classes):
    """Process model outputs with class information"""
    detections = []
    boxes = outputs[0][0]

    for box in boxes:
        # Get confidence and class scores
        confidence = box[4]
        if confidence > CONFIG['confidence_threshold']:
            # YOLO format: center_x, center_y, width, height, confidence, [class_scores]
            center_x, center_y, width, height = box[:4]

            # Get class probabilities and best class
            class_scores = box[5:5+num_classes]
            class_id = np.argmax(class_scores)
            class_score = class_scores[class_id]

            # Calculate geographic coordinates as before
            norm_x = center_x / CONFIG['model_input_size']
            norm_y = center_y / CONFIG['model_input_size']

            longitude = tile_bbox[0] + (norm_x * (tile_bbox[2] - tile_bbox[0]))
            latitude = tile_bbox[3] - (norm_y * (tile_bbox[3] - tile_bbox[1]))

            detections.append({
                'geometry': Point(longitude, latitude),
                'confidence': float(confidence),
                'class_id': int(class_id),
                'class_score': float(class_score),
                'bbox': [float(x) for x in box[:4]],
                'bbox_width': float(width),
                'bbox_height': float(height)
            })

    return detections

def improve_detection(img):
    """Simplified image preprocessing with most effective enhancements"""
    # Only keep the most effective enhancements
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    l_enhanced = clahe.apply(l)
    enhanced_lab = cv2.merge((l_enhanced, a, b))
    enhanced = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2RGB)

    return [img, enhanced]

def process_multiscale(wms, session, tile_bbox, num_classes, scales=[1.0]):
    """Simplified multiscale processing with fewer scales"""
    all_detections = []

    for scale in scales:
        try:
            original_size = CONFIG['tile_size_meters']
            scaled_size = original_size * scale
            center_lon = (tile_bbox[0] + tile_bbox[2]) / 2
            center_lat = (tile_bbox[1] + tile_bbox[3]) / 2
            size_lon, size_lat = meters_to_degrees(scaled_size, center_lat)

            scaled_bbox = (
                center_lon - size_lon/2,
                center_lat - size_lat/2,
                center_lon + size_lon/2,
                center_lat + size_lat/2
            )

            img = get_wms_image(wms, scaled_bbox)
            if img is None:
                continue

            img_array = np.array(img)
            enhanced_images = improve_detection(img_array)

            for enhanced in enhanced_images:
                input_tensor = cv2.cvtColor(enhanced, cv2.COLOR_RGB2BGR)
                input_tensor = np.transpose(input_tensor, (2, 0, 1))
                input_tensor = np.expand_dims(input_tensor, axis=0).astype(np.float32) / 255.0

                outputs = session.run(None, {session.get_inputs()[0].name: input_tensor})
                detections = process_detections(outputs, scaled_bbox, num_classes)

                for det in detections:
                    det['detection_scale'] = scale
                    all_detections.append(det)

        except Exception as e:
            logging.error(f"Error processing tile at scale {scale}: {str(e)}")
            continue

    return all_detections

def remove_duplicates(detections_gdf):
    """Duplicate removal with 2m threshold and progress bar"""
    if len(detections_gdf) == 0:
        return detections_gdf

    # Create UTM transformer for accurate distances
    proj_utm = pyproj.CRS('EPSG:32634')  # UTM zone 34N
    proj_wgs = pyproj.CRS('EPSG:4326')
    project = pyproj.Transformer.from_crs(proj_wgs, proj_utm, always_xy=True).transform

    # Convert to UTM
    detections_utm = detections_gdf.copy()
    detections_utm.geometry = detections_utm.geometry.apply(lambda geom: transform(project, geom))

    # Sort by confidence
    detections_utm = detections_utm.sort_values('confidence', ascending=False)

    kept_indices = []
    used_indices = set()
    distance_threshold = 2.0  # 2 meters

    # Add progress bar
    with tqdm(total=len(detections_utm), desc="Removing duplicates", leave=False) as pbar:
        for idx in detections_utm.index:
            if idx in used_indices:
                pbar.update(1)
                continue

            kept_indices.append(idx)
            point = detections_utm.loc[idx, 'geometry']

            # Find nearby points
            for other_idx in detections_utm.index:
                if other_idx != idx and other_idx not in used_indices:
                    other_point = detections_utm.loc[other_idx, 'geometry']
                    distance = point.distance(other_point)

                    if distance < distance_threshold:
                        used_indices.add(other_idx)

            pbar.update(1)

    # Convert back to WGS84
    project_back = pyproj.Transformer.from_crs(proj_utm, proj_wgs, always_xy=True).transform
    filtered_gdf = detections_gdf.loc[kept_indices].copy()
    filtered_gdf.geometry = filtered_gdf.geometry.apply(lambda geom: transform(project_back, transform(project, geom)))

    return filtered_gdf

def main():
    """Main execution with minimal console output"""
    try:
        paths = setup_environment()
        start_time = time.time()

        # Initialize components silently
        wms = WebMapService(CONFIG['wms_url'], version='1.3.0')
        session = ort.InferenceSession(paths['model_path'])
        num_classes = analyze_model(session)

        # Load and process frame
        frame_gdf = gpd.read_file(paths['frame_path'])
        if frame_gdf.crs.to_epsg() != 4326:
            frame_gdf = frame_gdf.to_crs(epsg=4326)

        bbox = frame_gdf.total_bounds
        tile_bboxes = get_tile_bboxes(bbox, CONFIG['tile_size_meters'])

        # Process tiles with clean progress bar
        all_detections = []
        with tqdm(total=len(tile_bboxes), desc="Processing tiles", unit="tile") as pbar:
            for tile_bbox in tile_bboxes:
                try:
                    detections = process_multiscale(wms, session, tile_bbox, num_classes, scales=[0.85, 1.0, 1.15])
                    if detections:
                        all_detections.extend(detections)
                except Exception as e:
                    # Silently log error to file and continue
                    logging.error(f"Tile processing error: {str(e)}")
                finally:
                    pbar.update(1)
                    gc.collect()

        # Create and process detections DataFrame
        if not all_detections:
            detections_gdf = gpd.GeoDataFrame(
                columns=['geometry', 'confidence', 'class_id', 'class_score', 'bbox'],
                crs="EPSG:4326"
            )
        else:
            detections_gdf = gpd.GeoDataFrame(all_detections, crs="EPSG:4326")

            # Show progress for duplicate removal
            with tqdm(total=1, desc="Removing duplicates", leave=True) as pbar:
                detections_gdf = remove_duplicates(detections_gdf)
                pbar.update(1)

        # Save results
        detections_gdf.to_file(paths['output_path'], driver='GeoJSON')

        # Final summary - just two lines
        minutes = (time.time() - start_time) / 60
        print(f"\nCompleted in {minutes:.1f} min. Found {len(detections_gdf)} objects.")

        return detections_gdf

    except Exception as e:
        print(f"\nError occurred. Check detection_debug.log for details.")
        logging.error(f"Processing failed: {str(e)}")
        return gpd.GeoDataFrame(
            columns=['geometry', 'confidence', 'class_id', 'class_score', 'bbox'],
            crs="EPSG:4326"
        )

if __name__ == "__main__":
    detections_gdf = main()

    # Print summary
    print(f"\nProcessing Summary:")
    # print(f"Total tiles processed: {len(tiles_gdf)}")
    print(f"Total detections: {len(detections_gdf)}")
    if len(detections_gdf) > 0:
        print(f"Average confidence: {detections_gdf['confidence'].mean():.3f}")

## GPU 1st attempt

In [ ]:
!pip uninstall -y onnxruntime
!pip install onnxruntime-gpu

In [ ]:
import onnxruntime as ort
print("Available Providers:", ort.get_available_providers())
print("ONNX Runtime Version:", ort.__version__)

In [ ]:
import onnxruntime as ort
import numpy as np
import cv2
from owslib.wms import WebMapService
import geopandas as gpd
from shapely.geometry import Point
import io
from PIL import Image
import os
from google.colab import drive
import torch
import concurrent.futures
import requests
from tqdm.auto import tqdm
import time
import gc
import math
from pyproj import Transformer
from shapely.ops import transform
import warnings
warnings.filterwarnings('ignore')

class CarDetector:
    def __init__(self):
        print("Initializing detector...")
        drive.mount('/content/drive', force_remount=True)
        self.base_dir = "/content/drive/Othercomputers/My_laptop/car_recognition"

        # Paths
        self.model_path = os.path.join(self.base_dir, "models/car_aerial_detection_yolo7_ITCVD_deepness.onnx")
        self.frame_path = os.path.join(self.base_dir, "gis/shp/frames/warsaw.shp")
        self.output_path = os.path.join(self.base_dir, "gis/shp/detection_results/warsaw.geojson")
        self.checkpoint_dir = os.path.join(self.base_dir, "checkpoints")
        os.makedirs(self.checkpoint_dir, exist_ok=True)

        # Configuration
        self.config = {
            'wms_url': "https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/StandardResolution",
            'tile_size_meters': 64.0,
            'pixel_size': 0.10,
            'confidence_threshold': 0.4,
            'tile_overlap': 0.2,
            'batch_size': 512,  # Increased for memory utilization
            'checkpoint_interval': 1000,
            'num_workers': 32,  # Increased for parallel fetching
            'queue_size': 512   # GPU memory queue size
        }

        self._setup_components()

    def _setup_components(self):
        """Initialize all components"""
        self._setup_gpu()
        self._setup_model()
        self._setup_wms()

    def _setup_gpu(self):
        """Configure GPU and CUDA"""
        if not torch.cuda.is_available():
            raise RuntimeError("CUDA is not available!")

        torch.cuda.empty_cache()
        torch.backends.cudnn.benchmark = True
        device = torch.cuda.current_device()
        print(f"\nGPU Info:")
        print(f"Device: {torch.cuda.get_device_name(device)}")
        print(f"Memory: {torch.cuda.get_device_properties(device).total_memory/1e9:.1f}GB")

    def _setup_model(self):
        """Initialize ONNX model with GPU optimization"""
        provider_options = {
            'device_id': 0,
            'gpu_mem_limit': int(14 * 1024 * 1024 * 1024),  # 14GB limit
            'arena_extend_strategy': 'kNextPowerOfTwo',
            'cudnn_conv_algo_search': 'EXHAUSTIVE'
        }

        providers = [('CUDAExecutionProvider', provider_options)]

        sess_options = ort.SessionOptions()
        sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
        sess_options.execution_mode = ort.ExecutionMode.ORT_PARALLEL
        sess_options.intra_op_num_threads = 4
        sess_options.inter_op_num_threads = 4

        self.session = ort.InferenceSession(
            self.model_path,
            sess_options=sess_options,
            providers=providers
        )
        print("Model loaded with GPU optimization")

    def _setup_wms(self):
        """Initialize WMS connection"""
        self.wms = WebMapService(self.config['wms_url'], version='1.3.0')
        print("WMS connection established")

    def get_wms_image(self, bbox):
        """Fetch single WMS image"""
        try:
            img = self.wms.getmap(
                layers=['Raster'],
                srs='EPSG:4326',
                bbox=bbox,
                size=(640, 640),
                format='image/jpeg',
                transparent=False
            )
            return Image.open(io.BytesIO(img.read())).convert('RGB')
        except Exception:
            return None

    def fetch_images_parallel(self, tile_bboxes):
        """Fetch multiple images in parallel"""
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.config['num_workers']) as executor:
            futures = [executor.submit(self.get_wms_image, bbox) for bbox in tile_bboxes]
            results = []
            for future, bbox in zip(futures, tile_bboxes):
                try:
                    img = future.result()
                    if img is not None:
                        results.append((img, bbox))
                except Exception:
                    continue
            return results

    def preprocess_image(self, img):
        """Preprocess image and move to GPU"""
        img_array = np.array(img)
        img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        tensor = torch.from_numpy(img_bgr).cuda()
        tensor = tensor.permute(2, 0, 1).float() / 255.0
        return tensor

    def process_gpu_queue(self, gpu_queue):
        """Process a queue of images stored in GPU memory"""
        detections = []

        for tensor, bbox in gpu_queue:
            try:
                # Prepare for model (add batch dimension)
                input_tensor = tensor.unsqueeze(0).cpu().numpy()

                # Run inference
                outputs = self.session.run(None, {self.session.get_inputs()[0].name: input_tensor})

                # Process detections
                boxes = outputs[0][0]
                conf_mask = boxes[:, 4] > self.config['confidence_threshold']
                boxes = boxes[conf_mask]

                if len(boxes) > 0:
                    # Move boxes to GPU for faster processing
                    boxes_tensor = torch.from_numpy(boxes).cuda()
                    centers = boxes_tensor[:, :2] / 640

                    # Calculate coordinates
                    lon_offset = bbox[2] - bbox[0]
                    lat_offset = bbox[3] - bbox[1]

                    lons = bbox[0] + (centers[:, 0] * lon_offset)
                    lats = bbox[3] - (centers[:, 1] * lat_offset)
                    confs = boxes_tensor[:, 4]

                    # Create detections
                    for lon, lat, conf in zip(lons.cpu().numpy(),
                                            lats.cpu().numpy(),
                                            confs.cpu().numpy()):
                        detections.append({
                            'geometry': Point(lon, lat),
                            'confidence': float(conf)
                        })

                    del boxes_tensor

            except Exception as e:
                print(f"Error processing queue item: {str(e)}")
                continue

        return detections

    def process_tile_batch(self, image_bbox_pairs):
        """Process a batch of images using GPU memory queue"""
        if not image_bbox_pairs:
            return []

        all_detections = []
        gpu_queue = []

        try:
            print(f"\nPreprocessing batch of {len(image_bbox_pairs)} images...")

            # Fill GPU queue
            for img, bbox in image_bbox_pairs:
                try:
                    tensor = self.preprocess_image(img)
                    gpu_queue.append((tensor, bbox))

                    # Process queue if full
                    if len(gpu_queue) >= self.config['queue_size']:
                        print(f"\nProcessing queue of {len(gpu_queue)} images...")
                        allocated = torch.cuda.memory_allocated() / 1e9
                        reserved = torch.cuda.memory_reserved() / 1e9
                        print(f"GPU Memory - Allocated: {allocated:.1f}GB, Reserved: {reserved:.1f}GB")

                        # Process current queue
                        queue_detections = self.process_gpu_queue(gpu_queue)
                        all_detections.extend(queue_detections)

                        # Clear queue
                        del gpu_queue[:]
                        gpu_queue = []
                        torch.cuda.empty_cache()

                except Exception as e:
                    print(f"Error preprocessing image: {str(e)}")
                    continue

            # Process remaining images
            if gpu_queue:
                print(f"\nProcessing remaining {len(gpu_queue)} images...")
                queue_detections = self.process_gpu_queue(gpu_queue)
                all_detections.extend(queue_detections)

            return all_detections

        except Exception as e:
            print(f"Batch processing error: {str(e)}")
            return []

        finally:
            # Clean up
            if 'gpu_queue' in locals():
                del gpu_queue
            torch.cuda.empty_cache()

    def generate_tiles(self, bounds):
        """Generate tile coordinates"""
        minx, miny, maxx, maxy = bounds
        mid_lat = (miny + maxy) / 2

        # Convert meters to degrees
        earth_radius = 6378137
        tile_meters = self.config['tile_size_meters']
        lat_deg = tile_meters / (earth_radius * math.pi / 180)
        lon_deg = tile_meters / (earth_radius * math.pi / 180 * math.cos(math.radians(mid_lat)))

        overlap = self.config['tile_overlap']
        step_lon = lon_deg * (1 - overlap)
        step_lat = lat_deg * (1 - overlap)

        tiles = []
        x = minx
        while x < maxx:
            y = miny
            while y < maxy:
                tiles.append((
                    x, y,
                    min(x + lon_deg, maxx),
                    min(y + lat_deg, maxy)
                ))
                y += step_lat
            x += step_lon

        return tiles

    def save_checkpoint(self, detections, checkpoint_num, processed_tiles):
        """Save detection results checkpoint"""
        if detections:
            checkpoint_path = os.path.join(
                self.checkpoint_dir,
                f"checkpoint_{checkpoint_num}_tiles_{processed_tiles}.geojson"
            )
            gdf = gpd.GeoDataFrame(detections, crs="EPSG:4326")
            gdf.to_file(checkpoint_path, driver='GeoJSON')
            print(f"\nSaved checkpoint {checkpoint_num} with {len(detections)} detections")

    def remove_duplicates(self, gdf):
        """Remove duplicate detections using spatial index"""
        if len(gdf) == 0:
            return gdf

        # Convert to UTM for accurate distance calculation
        transformer = Transformer.from_crs("EPSG:4326", "EPSG:32633", always_xy=True)
        gdf['geometry'] = gdf['geometry'].apply(lambda p: transform(transformer.transform, p))

        # Sort by confidence
        gdf = gdf.sort_values('confidence', ascending=False)

        # Remove duplicates
        kept_indices = []
        for idx in gdf.index:
            if idx in kept_indices:
                continue

            point = gdf.loc[idx, 'geometry']
            distances = gdf['geometry'].apply(lambda p: point.distance(p))
            duplicates = distances[distances < 2.0].index
            kept_indices.extend(duplicates)

        # Convert back to WGS84
        transformer = Transformer.from_crs("EPSG:32633", "EPSG:4326", always_xy=True)
        filtered_gdf = gdf.loc[kept_indices]
        filtered_gdf['geometry'] = filtered_gdf['geometry'].apply(
            lambda p: transform(transformer.transform, p)
        )

        return filtered_gdf

    def detect(self):
        """Main detection process with high GPU memory utilization"""
        try:
            # Load frame
            print("\nLoading frame...")
            frame_gdf = gpd.read_file(self.frame_path)
            if frame_gdf.crs.to_epsg() != 4326:
                frame_gdf = frame_gdf.to_crs(epsg=4326)

            bounds = frame_gdf.total_bounds

            # Generate tiles
            print("Generating tiles...")
            tiles = self.generate_tiles(bounds)
            total_tiles = len(tiles)
            print(f"Generated {total_tiles} tiles")

            # Initialize processing
            all_detections = []
            checkpoint_num = 0
            processed_count = 0

            # Process tiles in batches
            progress_bar = tqdm(total=total_tiles, desc="Processing tiles", unit="tiles")

            for start_idx in range(0, total_tiles, self.config['batch_size']):
                try:
                    # Get current batch
                    end_idx = min(start_idx + self.config['batch_size'], total_tiles)
                    current_batch = tiles[start_idx:end_idx]

                    # Fetch and process images
                    print(f"\nFetching batch of {len(current_batch)} images...")
                    image_bbox_pairs = self.fetch_images_parallel(current_batch)

                    if image_bbox_pairs:
                        # Process batch
                        batch_detections = self.process_tile_batch(image_bbox_pairs)
                        all_detections.extend(batch_detections)

                        # Update progress
                        batch_processed = len(current_batch)
                        processed_count += batch_processed
                        progress_bar.update(batch_processed)

                        # Print detailed status
                        print(f"\nBatch Statistics:")
                        print(f"Processed tiles: {processed_count}/{total_tiles}")
                        print(f"Current detections: {len(all_detections)}")
                        print(f"Batch detections: {len(batch_detections)}")

                        allocated = torch.cuda.memory_allocated() / 1e9
                        reserved = torch.cuda.memory_reserved() / 1e9
                        print(f"GPU Memory - Allocated: {allocated:.1f}GB, Reserved: {reserved:.1f}GB")

                        # Save checkpoint
                        if processed_count % self.config['checkpoint_interval'] == 0:
                            checkpoint_num += 1
                            self.save_checkpoint(all_detections, checkpoint_num, processed_count)

                    # Clean up memory
                    gc.collect()
                    torch.cuda.empty_cache()
                except Exception as e:
                    print(f"\nError processing batch: {str(e)}")
                    continue

            progress_bar.close()

            # Process final results
            if all_detections:
                print("\nProcessing final results...")
                detections_gdf = gpd.GeoDataFrame(all_detections, crs="EPSG:4326")
                print(f"Raw detections: {len(detections_gdf)}")

                print("Removing duplicates...")
                detections_gdf = self.remove_duplicates(detections_gdf)
                print(f"Final detections: {len(detections_gdf)}")

                detections_gdf.to_file(self.output_path, driver='GeoJSON')
                return detections_gdf

            return gpd.GeoDataFrame(columns=['geometry', 'confidence'], crs="EPSG:4326")

        except Exception as e:
            print(f"Error in detection process: {str(e)}")
            import traceback
            traceback.print_exc()
            return gpd.GeoDataFrame(columns=['geometry', 'confidence'], crs="EPSG:4326")

def main():
    try:
        # Initialize detector
        detector = CarDetector()

        # Run detection
        print("\nStarting detection process...")
        start_time = time.time()

        results = detector.detect()

        # Print final statistics
        elapsed_time = (time.time() - start_time) / 60  # Convert to minutes
        print("\nProcessing Complete!")
        print(f"Total time: {elapsed_time:.2f} minutes")

        if len(results) > 0:
            print(f"\nFinal Results:")
            print(f"Total detections: {len(results)}")
            print(f"Average confidence: {results['confidence'].mean():.3f}")
            print(f"Results saved to: {detector.output_path}")

            # Memory usage summary
            allocated = torch.cuda.memory_allocated() / 1e9
            reserved = torch.cuda.memory_reserved() / 1e9
            print(f"\nFinal GPU Memory Status:")
            print(f"Allocated: {allocated:.1f}GB")
            print(f"Reserved: {reserved:.1f}GB")

            return results
        else:
            print("No detections found")
            return None

    except Exception as e:
        print(f"Error in main process: {str(e)}")
        traceback.print_exc()
        return None
    finally:
        # Final cleanup
        torch.cuda.empty_cache()
        gc.collect()

if __name__ == "__main__":
    main()

## GPU 2nd attempt

In [2]:
!pip install onnxruntime-gpu owslib geopandas shapely pyproj tqdm pillow opencv-python-headless requests
!pip install psutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.5 MB/s eta 0:00:00


In [3]:
import onnxruntime as ort
import numpy as np
import cv2
from owslib.wms import WebMapService
import geopandas as gpd
from shapely.geometry import Point
import io
from PIL import Image
import os
from google.colab import drive
import torch
import concurrent.futures
import requests
from tqdm.auto import tqdm
import time
import gc
import math
import json
import psutil
from pyproj import Transformer
from shapely.ops import transform
import warnings
import traceback
warnings.filterwarnings('ignore')

class CarDetector:
    def __init__(self):
        print("Initializing detector...")
        drive.mount('/content/drive', force_remount=True)
        self.base_dir = "/content/drive/Othercomputers/My_laptop/car_recognition"

        # Create directory structure
        self.setup_directories()

        # Paths
        self.model_path = os.path.join(self.base_dir, "models/car_aerial_detection_yolo7_ITCVD_deepness.onnx")
        self.frame_path = os.path.join(self.base_dir, "gis/shp/frames/warsaw.shp")
        self.output_path = os.path.join(self.base_dir, "gis/shp/detection_results/warsaw.geojson")

        self.config = {
            'wms_url': "https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/StandardResolution",
            'tile_size_meters': 64.0,
            'confidence_threshold': 0.4,
            'tile_overlap': 0.1,
            'batch_size': 4096,          # Increased for faster processing
            'checkpoint_interval': 5000,
            'num_workers': 32,
            'queue_size': 1024,          # Increased for better GPU utilization
            'max_gpu_memory': 12.0,
            'min_gpu_memory': 8.0,
            'ram_chunk_size': 512,       # Increased for better RAM utilization
            'duplicate_distance': 2.0     # Added missing parameter (meters)
        }

        # Initialize checkpoint files
        self.initialize_checkpoints()

        print(f"\nConfiguration:")
        print(f"- GPU target memory: {self.config['min_gpu_memory']}-{self.config['max_gpu_memory']}GB")
        print(f"- Batch size: {self.config['batch_size']}")
        print(f"- Queue size: {self.config['queue_size']}")
        print(f"- Workers: {self.config['num_workers']}")
        print(f"- Checkpoint interval: {self.config['checkpoint_interval']} tiles")

        self._setup_components()

    def setup_directories(self):
        """Create necessary directories and verify their existence"""
        # Create main directories
        directories = [
            os.path.join(self.base_dir, "checkpoints"),
            os.path.join(self.base_dir, "models"),
            os.path.join(self.base_dir, "gis/shp/detection_results"),
        ]

        for directory in directories:
            os.makedirs(directory, exist_ok=True)
            if os.path.exists(directory):
                print(f"Verified directory: {directory}")
            else:
                raise RuntimeError(f"Failed to create directory: {directory}")

        self.checkpoint_dir = directories[0]

        # Set checkpoint paths
        self.checkpoint_state = os.path.join(self.checkpoint_dir, "processing_state.json")
        self.checkpoint_data = os.path.join(self.checkpoint_dir, "latest_detections.geojson")

    def initialize_checkpoints(self):
        """Initialize checkpoint files only if they don't exist"""
        try:
            # Only create checkpoint files if they don't exist
            if not os.path.exists(self.checkpoint_state):
                # Create empty checkpoint state
                initial_state = {
                    'initialized': time.time(),
                    'processed_tiles': 0,
                    'total_tiles': 0,
                    'last_processed_index': -1
                }

                with open(self.checkpoint_state, 'w') as f:
                    json.dump(initial_state, f)

            if not os.path.exists(self.checkpoint_data):
                # Create empty GeoDataFrame for detections
                empty_gdf = gpd.GeoDataFrame(columns=['geometry', 'confidence'], crs="EPSG:4326")
                empty_gdf.to_file(self.checkpoint_data, driver='GeoJSON')

            # Verify files exist
            if os.path.exists(self.checkpoint_state) and os.path.exists(self.checkpoint_data):
                print(f"\nCheckpoint files verified:")
                print(f"- State: {self.checkpoint_state}")
                print(f"- Data: {self.checkpoint_data}")
            else:
                raise RuntimeError("Failed to verify checkpoint files")

        except Exception as e:
            print(f"Error initializing checkpoints: {str(e)}")
            raise

    def _setup_components(self):
        """Initialize processing components"""
        self._setup_gpu()
        self._setup_model()
        self._setup_wms()

    def _setup_gpu(self):
        """Configure GPU and CUDA"""
        if not torch.cuda.is_available():
            raise RuntimeError("CUDA is not available!")

        torch.cuda.empty_cache()
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.fastest = True
        device = torch.cuda.current_device()

        print(f"\nGPU Information:")
        print(f"Device: {torch.cuda.get_device_name(device)}")
        print(f"Memory: {torch.cuda.get_device_properties(device).total_memory/1e9:.1f}GB")

        # Warm up GPU
        dummy = torch.zeros(1, 3, 640, 640, device='cuda')
        del dummy
        torch.cuda.empty_cache()

    def _setup_model(self):
        """Initialize ONNX model with GPU optimization"""
        provider_options = {
            'device_id': 0,
            'gpu_mem_limit': int(self.config['max_gpu_memory'] * 1024 * 1024 * 1024),
            'arena_extend_strategy': 'kNextPowerOfTwo',
            'cudnn_conv_algo_search': 'EXHAUSTIVE',
            'do_copy_in_default_stream': True
        }

        providers = [('CUDAExecutionProvider', provider_options)]

        sess_options = ort.SessionOptions()
        sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
        sess_options.execution_mode = ort.ExecutionMode.ORT_PARALLEL
        sess_options.intra_op_num_threads = 4
        sess_options.inter_op_num_threads = 4

        self.session = ort.InferenceSession(
            self.model_path,
            sess_options=sess_options,
            providers=providers
        )
        print("Model loaded with GPU optimization")

    def _setup_wms(self):
        """Initialize WMS connection"""
        self.wms = WebMapService(self.config['wms_url'], version='1.3.0')
        print("WMS connection established")

    def get_wms_image(self, bbox):
        """Fetch single WMS image with timeout"""
        try:
            img = self.wms.getmap(
                layers=['Raster'],
                srs='EPSG:4326',
                bbox=bbox,
                size=(640, 640),
                format='image/jpeg',
                transparent=False,
                timeout=30  # Add timeout
            )
            return Image.open(io.BytesIO(img.read())).convert('RGB')
        except Exception:
            return None

    def fetch_images_parallel(self, tile_bboxes):
        """Fetch images with parallel processing and timeout"""
        results = []
        futures = []

        with concurrent.futures.ThreadPoolExecutor(max_workers=self.config['num_workers']) as executor:
            # Submit all tasks
            for bbox in tile_bboxes:
                futures.append((executor.submit(self.get_wms_image, bbox), bbox))

            # Process completed tasks
            for future, bbox in futures:
                try:
                    img = future.result(timeout=30)  # Add timeout for each task
                    if img is not None:
                        results.append((img, bbox))
                except (concurrent.futures.TimeoutError, Exception) as e:
                    continue

        return results

    def preprocess_image(self, img):
        """Preprocess image and move to GPU"""
        img_array = np.array(img)
        img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        tensor = torch.from_numpy(img_bgr).cuda()
        tensor = tensor.to(dtype=torch.float32)
        tensor = tensor.permute(2, 0, 1) / 255.0
        return tensor

    def process_gpu_queue(self, gpu_queue):
        """Process queue of images in GPU memory"""
        detections = []

        for tensor, bbox in gpu_queue:
            try:
                # Add batch dimension and run inference
                input_tensor = tensor.unsqueeze(0).cpu().numpy()
                outputs = self.session.run(None, {self.session.get_inputs()[0].name: input_tensor})

                boxes = outputs[0][0]
                conf_mask = boxes[:, 4] > self.config['confidence_threshold']
                boxes = boxes[conf_mask]

                if len(boxes) > 0:
                    # Move boxes to GPU for faster processing
                    boxes_tensor = torch.from_numpy(boxes).cuda()
                    centers = boxes_tensor[:, :2] / 640

                    lon_offset = bbox[2] - bbox[0]
                    lat_offset = bbox[3] - bbox[1]

                    lons = bbox[0] + (centers[:, 0] * lon_offset)
                    lats = bbox[3] - (centers[:, 1] * lat_offset)
                    confs = boxes_tensor[:, 4]

                    for lon, lat, conf in zip(lons.cpu().numpy(),
                                            lats.cpu().numpy(),
                                            confs.cpu().numpy()):
                        detections.append({
                            'geometry': Point(lon, lat),
                            'confidence': float(conf)
                        })

                    del boxes_tensor

            except Exception:
                continue

        return detections

    def process_tile_batch(self, image_bbox_pairs):
        """Process batch with GPU optimization"""
        if not image_bbox_pairs:
            return []

        all_detections = []
        try:
            # Convert and move all images to GPU first
            gpu_tensors = []
            for img, bbox in image_bbox_pairs:
                try:
                    tensor = self.preprocess_image(img)
                    gpu_tensors.append((tensor, bbox))
                except Exception:
                    continue

            # Process in smaller sub-batches to maintain GPU memory
            for i in range(0, len(gpu_tensors), self.config['queue_size']):
                sub_batch = gpu_tensors[i:i + self.config['queue_size']]
                batch_detections = self.process_gpu_queue(sub_batch)
                all_detections.extend(batch_detections)

            return all_detections

        except Exception as e:
            print(f"Batch processing error: {str(e)}")
            return []
        finally:
            # Clean up GPU memory
            if 'gpu_tensors' in locals():
                del gpu_tensors
            torch.cuda.empty_cache()
            gc.collect()

    def remove_duplicates(self, detections):
        """Remove duplicate detections efficiently"""
        if not detections:
            return []

        try:
            gdf = gpd.GeoDataFrame(detections, crs="EPSG:4326")

            # Convert to UTM for accurate distance calculation
            transformer = Transformer.from_crs("EPSG:4326", "EPSG:32633", always_xy=True)
            gdf['geometry'] = gdf['geometry'].apply(lambda p: transform(transformer.transform, p))

            # Sort by confidence
            gdf = gdf.sort_values('confidence', ascending=False)

            # Remove duplicates
            kept_indices = []
            for idx in gdf.index:
                if idx in kept_indices:
                    continue

                point = gdf.loc[idx, 'geometry']
                distances = gdf['geometry'].apply(lambda p: point.distance(p))
                duplicates = distances[distances < self.config['duplicate_distance']].index
                kept_indices.extend(duplicates)

            # Convert back to WGS84
            transformer = Transformer.from_crs("EPSG:32633", "EPSG:4326", always_xy=True)
            filtered_gdf = gdf.loc[kept_indices]
            filtered_gdf['geometry'] = filtered_gdf['geometry'].apply(
                lambda p: transform(transformer.transform, p)
            )

            return filtered_gdf.to_dict('records')

        except Exception as e:
            print(f"Deduplication error: {str(e)}")
            return detections  # Return original if deduplication fails

    def save_checkpoint(self, detections, processed_tiles, total_tiles):
        """Save checkpoint with processing state"""
        try:
            # Save processing state
            checkpoint_data = {
                'processed_tiles': processed_tiles,
                'total_tiles': total_tiles,
                'last_processed_index': processed_tiles - 1,
                'timestamp': time.time()
            }

            with open(self.checkpoint_state, 'w') as f:
                json.dump(checkpoint_data, f)

            # Save detections
            unique_detections = self.remove_duplicates(detections)
            gdf = gpd.GeoDataFrame(unique_detections, crs="EPSG:4326")
            gdf.to_file(self.checkpoint_data, driver='GeoJSON')

            print(f"\nCheckpoint saved at {processed_tiles}/{total_tiles} tiles")
            print(f"Stored {len(unique_detections)} unique detections")

        except Exception as e:
            print(f"Checkpoint save error: {str(e)}")

    def load_checkpoint(self):
        """Load previous checkpoint if exists"""
        if os.path.exists(self.checkpoint_state) and os.path.exists(self.checkpoint_data):
            try:
                with open(self.checkpoint_state, 'r') as f:
                    checkpoint_data = json.load(f)

                gdf = gpd.read_file(self.checkpoint_data)
                detections = gdf.to_dict('records')

                print(f"\nFound checkpoint:")
                print(f"- Processed: {checkpoint_data['processed_tiles']} tiles")
                print(f"- Detections: {len(detections)}")
                print(f"- Time: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(checkpoint_data['timestamp']))}")

                response = input("\nResume from checkpoint? (y/n): ").lower()
                if response == 'y':
                    return checkpoint_data, detections
                else:
                    print("Starting fresh processing...")
                    os.remove(self.checkpoint_state)
                    os.remove(self.checkpoint_data)

            except Exception as e:
                print(f"Error loading checkpoint: {str(e)}")

        return None, []

    def generate_tiles(self, bounds):
        """Generate tile coordinates"""
        minx, miny, maxx, maxy = bounds
        mid_lat = (miny + maxy) / 2

        # Convert meters to degrees
        earth_radius = 6378137
        tile_meters = self.config['tile_size_meters']
        lat_deg = tile_meters / (earth_radius * math.pi / 180)
        lon_deg = tile_meters / (earth_radius * math.pi / 180 * math.cos(math.radians(mid_lat)))

        overlap = self.config['tile_overlap']
        step_lon = lon_deg * (1 - overlap)
        step_lat = lat_deg * (1 - overlap)

        tiles = []
        x = minx
        while x < maxx:
            y = miny
            while y < maxy:
                tiles.append((
                    x, y,
                    min(x + lon_deg, maxx),
                    min(y + lat_deg, maxy)
                ))
                y += step_lat
            x += step_lon

        return tiles

    def detect(self):
        """Main detection process with optimized performance"""
        try:
            start_time = time.time()

            # Load frame
            frame_gdf = gpd.read_file(self.frame_path)
            if frame_gdf.crs.to_epsg() != 4326:
                frame_gdf = frame_gdf.to_crs(epsg=4326)

            # Generate tiles
            tiles = self.generate_tiles(frame_gdf.total_bounds)
            total_tiles = len(tiles)
            del frame_gdf
            gc.collect()

            # Load checkpoint or initialize
            checkpoint_data, all_detections = self.load_checkpoint()
            if checkpoint_data:
                processed_count = checkpoint_data['processed_tiles']
                start_idx = checkpoint_data['last_processed_index'] + 1
            else:
                processed_count = 0
                start_idx = 0
                all_detections = []

            print(f"\nProcessing {total_tiles} tiles...")
            progress_bar = tqdm(total=total_tiles, initial=processed_count, desc="Processing", unit="tiles")

            last_checkpoint = processed_count

            # Process tiles in batches
            for idx in range(start_idx, total_tiles, self.config['batch_size']):
                try:
                    # Get current batch
                    end_idx = min(idx + self.config['batch_size'], total_tiles)
                    current_batch = tiles[idx:end_idx]

                    # Process batch
                    image_bbox_pairs = self.fetch_images_parallel(current_batch)

                    if image_bbox_pairs:
                        batch_detections = self.process_tile_batch(image_bbox_pairs)

                        # Manage memory for detections
                        if len(all_detections) > 100000:
                            all_detections = self.remove_duplicates(all_detections)

                        all_detections.extend(batch_detections)

                        # Update progress
                        batch_processed = len(current_batch)
                        processed_count += batch_processed
                        progress_bar.update(batch_processed)

                        # Check for checkpoint
                        if processed_count - last_checkpoint >= self.config['checkpoint_interval']:
                            self.save_checkpoint(all_detections, processed_count, total_tiles)
                            last_checkpoint = processed_count

                        # Print status every 10k tiles
                        if processed_count % 10000 == 0:
                            elapsed = (time.time() - start_time) / 60
                            remaining = (elapsed / (processed_count - start_idx + 1)) * (total_tiles - processed_count)
                            print(f"\nStatus Update:")
                            print(f"Time: {elapsed:.1f}min elapsed, ~{remaining:.1f}min remaining")
                            print(f"Memory: {psutil.Process().memory_info().rss/1e9:.1f}GB RAM")
                            print(f"GPU: {torch.cuda.memory_allocated()/1e9:.1f}GB")
                            print(f"Detections: {len(batch_detections)} in batch, {len(all_detections)} total")

                    # Clean up
                    del image_bbox_pairs
                    gc.collect()
                    torch.cuda.empty_cache()

                except Exception as e:
                    print(f"\nBatch error: {str(e)}")
                    self.save_checkpoint(all_detections, processed_count, total_tiles)
                    continue

            progress_bar.close()

            # Continuing from detect() method
            # Process final results
            if all_detections:
                print("\nProcessing final results...")
                final_detections = self.remove_duplicates(all_detections)
                results_gdf = gpd.GeoDataFrame(final_detections, crs="EPSG:4326")

                print(f"Saving {len(results_gdf)} final detections...")
                results_gdf.to_file(self.output_path, driver='GeoJSON')

                total_time = (time.time() - start_time) / 60
                print(f"\nProcessing Complete!")
                print(f"Time: {total_time:.1f} minutes")
                print(f"Tiles: {total_tiles}")
                print(f"Detections: {len(results_gdf)}")
                print(f"Results saved to: {self.output_path}")

                # Clean up checkpoint files on successful completion
                if os.path.exists(self.checkpoint_state):
                    os.remove(self.checkpoint_state)
                if os.path.exists(self.checkpoint_data):
                    os.remove(self.checkpoint_data)

                return results_gdf

            return gpd.GeoDataFrame(columns=['geometry', 'confidence'], crs="EPSG:4326")

        except Exception as e:
            print(f"\nError in detection process: {str(e)}")
            traceback.print_exc()
            # Try to save checkpoint on error
            try:
                self.save_checkpoint(all_detections, processed_count, total_tiles)
                print("Checkpoint saved after error - you can resume later")
            except:
                print("Failed to save checkpoint after error")
            return gpd.GeoDataFrame(columns=['geometry', 'confidence'], crs="EPSG:4326")
        finally:
            # Final cleanup
            torch.cuda.empty_cache()
            gc.collect()

def main():
    """Main execution with error handling"""
    try:
        # Initialize detector
        detector = CarDetector()

        # Run detection
        print("\nStarting detection process...")
        start_time = time.time()

        results = detector.detect()

        if len(results) > 0:
            print(f"\nFinal Statistics:")
            print(f"- Total detections: {len(results)}")
            print(f"- Average confidence: {results['confidence'].mean():.3f}")

            # Memory usage summary
            ram_usage = psutil.Process().memory_info().rss / 1e9
            gpu_allocated = torch.cuda.memory_allocated() / 1e9
            gpu_reserved = torch.cuda.memory_reserved() / 1e9

            print(f"\nFinal Memory Usage:")
            print(f"- RAM: {ram_usage:.1f}GB")
            print(f"- GPU Allocated: {gpu_allocated:.1f}GB")
            print(f"- GPU Reserved: {gpu_reserved:.1f}GB")

            total_time = (time.time() - start_time) / 60
            print(f"\nTotal processing time: {total_time:.1f} minutes")
            print(f"Average speed: {len(results)/total_time:.1f} detections/minute")

            return results
        else:
            print("No detections found")
            return None

    except Exception as e:
        print(f"Error in main process: {str(e)}")
        traceback.print_exc()
        return None
    finally:
        # Final cleanup
        torch.cuda.empty_cache()
        gc.collect()

if __name__ == "__main__":
    main()

Initializing detector...
Mounted at /content/drive
Verified directory: /content/drive/Othercomputers/My_laptop/car_recognition/checkpoints
Verified directory: /content/drive/Othercomputers/My_laptop/car_recognition/models
Verified directory: /content/drive/Othercomputers/My_laptop/car_recognition/gis/shp/detection_results

Checkpoint files verified:
- State: /content/drive/Othercomputers/My_laptop/car_recognition/checkpoints/processing_state.json
- Data: /content/drive/Othercomputers/My_laptop/car_recognition/checkpoints/latest_detections.geojson

Configuration:
- GPU target memory: 8.0-12.0GB
- Batch size: 4096
- Queue size: 1024
- Workers: 32
- Checkpoint interval: 5000 tiles

GPU Information:
Device: Tesla T4
Memory: 15.8GB
Model loaded with GPU optimization
Error in main process: HTTPSConnectionPool(host='mapy.geoportal.gov.pl', port=443): Max retries exceeded with url: /wss/service/PZGIK/ORTO/WMS/StandardResolution?service=WMS&request=GetCapabilities&version=1.3.0 (Caused by NameR

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 199, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python3.10/socket.py", line 955, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 490, in _make_request
    raise new_e
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 466, in _make_